# Pipeline Notebook

---
## Train a Classifier - SVM

In [ ]:
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

### Load Training Data

In [ ]:
CAR_DIR_REGEX = '../dataset/vehicles/**/*.png'
NOCAR_DIR_REGEX = '../dataset/non-vehicles/**/*.png'

car_fnames = glob.glob(CAR_DIR_REGEX, recursive=True)
nocar_fnames = glob.glob(NOCAR_DIR_REGEX, recursive=True)

print('Loading data...')
print()
car_imgs = np.array([mpimg.imread(fname) for fname in car_fnames])
nocar_imgs = np.array([mpimg.imread(fname) for fname in nocar_fnames])

car_imgs_n = car_imgs.shape[0]
nocar_imgs_n = nocar_imgs.shape[0]

print('Loaded {} images of cars.'.format(car_imgs_n))
print('Loaded {} images of non-cars.'.format(nocar_imgs_n))
print()
print('Single image has size {}.'.format(car_imgs.shape[1:3]))
print('Single image has {} colors.'.format(car_imgs.shape[3]))

### Explore Training Data

In [ ]:
def display_images(images, title=None, limit=16, need_shuffle=False):
    if need_shuffle==True:
        images = shuffle(images, random_state=0)

    img_count = limit if (len(images)>limit) else len(images)
    m_cols = 8
    m_rows = int(img_count/m_cols) + (int(img_count/m_cols)==0)

    plt.figure(figsize=(15,4))
    for idx in range(limit):
        plt.subplot(m_rows, m_cols, idx+1)
        plt.axis('off')
        if title is not None: plt.title(title[idx])
        plt.imshow(images[idx])

    plt.show()
    return

print('Cars')
display_images(car_imgs, need_shuffle=True)
print('No-Cars')
display_images(nocar_imgs, need_shuffle=True)

In [ ]:
def display_hist_classes(cars_images, nocars_images):
    xticks = np.arange(4)
    ind = xticks[1:3]
    width = 0.65

    class_counts = (len(cars_images), len(nocars_images))
    plt.bar(ind, class_counts, width, facecolor='green', alpha=0.5, align='center')
    plt.xticks(xticks, ('', 'Cars', 'No-Cars', ''))
    plt.yticks(np.arange(0, int(1.2*max(class_counts)), 1000))
    plt.grid(True)
    plt.show()
    return

display_hist_classes(car_imgs, nocar_imgs)

### Define Training Features

In [ ]:
# Choose two samples randomly
titles = ['Car', 'No-Car']
car_img = car_imgs[np.random.randint(car_imgs_n)]
nocar_img = nocar_imgs[np.random.randint(nocar_imgs_n)]


display_images([car_img, nocar_img], title=titles, limit=2)

### Color Features

In [ ]:
def compute_color_hist(img, color_space='RGB', nbins=32, bins_range=(0, 256)):
    """ Computes the histograms of the given image on all of the 3 channels.
        Eventually, it converts the image to a new color space.
        Before computing the histogram, the pixel values are
        scaled to match the bins_range.
    """
    channels = img.shape[2]
    w, h = img.shape[1], car_img.shape[0]
    scaler = MinMaxScaler(feature_range=bins_range, copy=False)

    # color space convertion
    if color_space == 'HSV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    elif color_space == 'HLS':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    elif color_space == 'LUV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    elif color_space == 'YUV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    else:
        conv_img = np.copy(img)

    histograms = []
    for idx in range(channels):
        # get the channel idx and reshape
        # the array for the scaler
        ch = conv_img[:,:,idx].reshape(w*h,1)
        # scale the values to the given range
        ch_norm = scaler.fit_transform(ch)
        # compute the histogram
        ch_hist = np.histogram(ch_norm, bins=nbins, range=bins_range)
        # append the histogram
        histograms.append(ch_hist)

    return np.array(histograms)

def compare_color_hists(car_histograms, nocar_histograms, ch_names=['Red','Green','Blue']):
    """ Plots the histogram of each image channel in
        such a way to enhance Car and No-Car color
        channel features.
    """
    channels = car_histograms.shape[0]

    plt.figure(figsize=(15,4))
    for idx in range(channels):
        plt.subplot(1, 3, idx+1)
        if ch_names is not None: plt.title(ch_names[idx])

        # compute bar center using bin ranges
        x_ticks = car_histograms[idx][1][1:] - car_histograms[idx][1][1]/2
        car_data = plt.plot(x_ticks, car_histograms[idx][0], 'b', label='Car')
        nocar_data = plt.plot(x_ticks, nocar_histograms[idx][0], 'r', label='No-Car')

        plt.legend()
        plt.tight_layout()

    plt.show()
    return

In [ ]:
# RGB
rgb_car_hists = compute_color_hist(car_img)
rgb_nocar_hists = compute_color_hist(nocar_img)
compare_color_hists(rgb_car_hists, rgb_nocar_hists)

# HSV
hsv_car_hists = compute_color_hist(car_img, color_space='HSV')
hsv_nocar_hists = compute_color_hist(nocar_img, color_space='HSV')
compare_color_hists(hsv_car_hists, hsv_nocar_hists, ch_names=['Hue','Saturation','Value'])

# HSL
hls_car_hists = compute_color_hist(car_img, color_space='HLS')
hls_nocar_hists = compute_color_hist(nocar_img, color_space='HLS')
compare_color_hists(hls_car_hists, hls_nocar_hists, ch_names=['Hue','Lightness','Saturation'])

### Gradient Features

In [ ]:
def compute_hog(img, pix_per_cell=8, cell_per_block=2, orient=9, vis=False, feature_vec=True):
    """ Computes the Histogram of Oriented Gradient.
        The total number of features in your final feature vector
        will be the total number of block positions multiplied by
        the number of cells per block, times the number of orientations.
    
    img: a single color channel or grayscaled image.
    
    pix_per_cell: specifies the cell size over which each gradient
        histogram is computed. This paramater is passed as a 2-tuple
        but cells are commonly chosen to be square.

    cell_per_block: passed as a 2-tuple, and specifies the local area
        over which the histogram counts in a given cell will be normalized.
        Block normalization is not necessarily required, but generally
        leads to a more robust feature set.
    
    orient: specified as an integer, and represents the number of
        orientation bins that the gradient information will be split
        up into in the histogram. Typical values are between 6 and 12 bins.

    vis: if True, the function returns an image as 2nd parameter
    
    feature_vec: if True, returns the HOG for the image as 1D (flattened) array.
    """
    if vis == True:
        features, hog_img = hog(img,
                                orientations=orient,
                                pixels_per_cell=(pix_per_cell, pix_per_cell),
                                cells_per_block=(cell_per_block, cell_per_block),
                                transform_sqrt=False, 
                                visualise=True,
                                feature_vector=False)
        return features, hog_img
    else:      
        features = hog(img,
                       orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False, 
                       visualise=False,
                       feature_vector=feature_vec)
        return features

def compare_hog_hists(car_histogram, nocar_histogram):
    plt.figure(figsize=(10,4))

    plt.subplot(1, 2, 1)
    plt.title("Car HOG")
    plt.imshow(car_histogram)

    plt.subplot(1, 2, 2)
    plt.title("No-Car HOG")
    plt.imshow(nocar_histogram)

    plt.show()
    return

In [ ]:
# From the color features analysis we have choosen HSV S-channel
car_hsv_img = cv2.cvtColor(car_img, cv2.COLOR_RGB2HSV)
nocar_hsv_img = cv2.cvtColor(nocar_img, cv2.COLOR_RGB2HSV)
car_s_ch = car_hsv_img[:,:,1]
nocar_s_ch = nocar_hsv_img[:,:,1]

# Compute HOG
car_features, car_hog_img = compute_hog(car_s_ch, vis=True, feature_vec=False)
nocar_features, nocar_hog_img = compute_hog(nocar_s_ch, vis=True, feature_vec=False)

# Display HOG
compare_hog_hists(car_hog_img, nocar_hog_img)

### Train Test Split

### Training

### Evaluation

---
## Sliding Window Search

### Define Searching Area

### Searching

### Identifiy Cars Position - Centroid Method

---
## Pipeline - Video